# Laboratorium

### Konfiguracja

In [45]:
# Narzędzie jest oparte o kilka zewnętrznych bibliotek, które potrzebujemy najpierw zaimportować.
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
from matplotlib.widgets import Button
import json as js



# Parametr określający jak blisko (w odsetku całego widocznego zakresu) punktu początkowego 
# wielokąta musimy kliknąć, aby go zamknąć.
TOLERANCE = 0.15

%matplotlib notebook

def dist(point1, point2):
    return np.sqrt(np.power(point1[0] - point2[0], 2) + np.power(point1[1] - point2[1], 2))


# Klasa ta trzyma obecny stan wykresu oraz posiada metody, które mają zostać wykonane
# po naciśnięciu przycisków.
class _Button_callback(object):
    def __init__(self, scenes):
        self.i = 0
        self.scenes = scenes
        self.adding_points = False
        self.added_points = []
        self.adding_lines = False
        self.added_lines = []
        self.adding_rects = False
        self.added_rects = []

    def set_axes(self, ax):
        self.ax = ax

    # Metoda ta obsługuje logikę przejścia do następnej sceny.
    def next(self, event):
        self.i = (self.i + 1) % len(self.scenes)
        self.draw(autoscaling=True)

    # Metoda ta obsługuje logikę powrotu do poprzedniej sceny.
    def prev(self, event):
        self.i = (self.i - 1) % len(self.scenes)
        self.draw(autoscaling=True)

    # Metoda ta aktywuje funkcję rysowania punktów wyłączając równocześnie rysowanie 
    # odcinków i wielokątów.
    def add_point(self, event):
        self.adding_points = not self.adding_points
        self.new_line_point = None
        if self.adding_points:
            self.adding_lines = False
            self.adding_rects = False
            self.added_points.append(PointsCollection([]))

    # Metoda ta aktywuje funkcję rysowania odcinków wyłączając równocześnie
    # rysowanie punktów i wielokątów.     
    def add_line(self, event):
        self.adding_lines = not self.adding_lines
        self.new_line_point = None
        if self.adding_lines:
            self.adding_points = False
            self.adding_rects = False
            self.added_lines.append(LinesCollection([]))

    # Metoda ta aktywuje funkcję rysowania wielokątów wyłączając równocześnie
    # rysowanie punktów i odcinków.
    def add_rect(self, event):
        self.adding_rects = not self.adding_rects
        self.new_line_point = None
        if self.adding_rects:
            self.adding_points = False
            self.adding_lines = False
            self.new_rect()

    def new_rect(self):
        self.added_rects.append(LinesCollection([]))
        self.rect_points = []

    # Metoda odpowiedzialna za właściwą logikę rysowania nowych elementów. W
    # zależności od włączonego trybu dodaje nowe punkty, początek, koniec odcinka
    # lub poszczególne wierzchołki wielokąta. Istnieje ciekawa logika sprawdzania
    # czy dany punkt jest domykający dla danego wielokąta. Polega ona na tym, że
    # sprawdzamy czy odległość nowego punktu od początkowego jest większa od
    # średniej długości zakresu pomnożonej razy parametr TOLERANCE.   
    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        new_point = (event.xdata, event.ydata)
        if self.adding_points:
            self.added_points[-1].add_points([new_point])
            self.draw(autoscaling=False)
        elif self.adding_lines:
            if self.new_line_point is not None:
                self.added_lines[-1].add([self.new_line_point, new_point])
                self.new_line_point = None
                self.draw(autoscaling=False)
            else:
                self.new_line_point = new_point
        elif self.adding_rects:
            if len(self.rect_points) == 0:
                self.rect_points.append(new_point)
            elif len(self.rect_points) == 1:
                self.added_rects[-1].add([self.rect_points[-1], new_point])
                self.rect_points.append(new_point)
                self.draw(autoscaling=False)
            elif len(self.rect_points) > 1:
                if dist(self.rect_points[0], new_point) < (
                        np.mean([self.ax.get_xlim(), self.ax.get_ylim()]) * TOLERANCE):
                    self.added_rects[-1].add([self.rect_points[-1], self.rect_points[0]])
                    self.new_rect()
                else:
                    self.added_rects[-1].add([self.rect_points[-1], new_point])
                    self.rect_points.append(new_point)
                self.draw(autoscaling=False)

    # Metoda odpowiedzialna za narysowanie całego wykresu. Warto zauważyć,
    # że zaczyna się ona od wyczyszczenia jego wcześniejszego stanu. Istnieje w
    # niej nietrywialna logika zarządzania zakresem wykresu, tak żeby, w zależności
    # od ustawionego parametru autoscaling, uniknąć sytuacji, kiedy dodawanie
    # nowych punktów przy brzegu obecnie widzianego zakresu powoduje niekorzystne
    # przeskalowanie.
    def draw(self, autoscaling=True):
        if not autoscaling:
            xlim = self.ax.get_xlim()
            ylim = self.ax.get_ylim()
        self.ax.clear()
        for collection in (self.scenes[self.i].points + self.added_points):
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), **collection.kwargs)
        for collection in (self.scenes[self.i].lines + self.added_lines + self.added_rects):
            self.ax.add_collection(collection.get_collection())
        self.ax.autoscale(autoscaling)
        if not autoscaling:
            self.ax.set_xlim(xlim)
            self.ax.set_ylim(ylim)
        plt.draw()


### Interfejsy

[Dostępne kolory](https://matplotlib.org/3.1.1/gallery/color/named_colors.html)

[Dostępne znaczniki punktów](https://matplotlib.org/3.1.1/api/markers_api.html#module-matplotlib.markers)

In [30]:
# Klasa Scene odpowiada za przechowywanie elementów, które mają być
# wyświetlane równocześnie. Konkretnie jest to lista PointsCollection i
# LinesCollection.
class Scene:
    def __init__(self, points=[], lines=[]):
        self.points = points
        self.lines = lines


# Klasa PointsCollection gromadzi w sobie punkty jednego typu, a więc takie,
# które zostaną narysowane w takim samym kolorze i stylu. W konstruktorze
# przyjmuje listę punktów rozumianych jako pary współrzędnych (x, y). Parametr
# kwargs jest przekazywany do wywołania funkcji z biblioteki MatPlotLib przez
# co użytkownik może podawać wszystkie parametry tam zaproponowane.        
class PointsCollection:
    def __init__(self, points, **kwargs):
        self.points = points
        self.kwargs = kwargs

    def add_points(self, points):
        self.points = self.points + points


# Klasa LinesCollection podobnie jak jej punktowy odpowiednik gromadzi
# odcinki tego samego typu. Tworząc ją należy podać listę linii, gdzie każda
# z nich jest dwuelementową listą punktów – par (x, y). Parametr kwargs jest
# przekazywany do wywołania funkcji z biblioteki MatPlotLib przez co użytkownik
# może podawać wszystkie parametry tam zaproponowane.
class LinesCollection:
    def __init__(self, lines, **kwargs):
        self.lines = lines
        self.kwargs = kwargs

    def add(self, line):
        self.lines.append(line)

    def get_collection(self):
        return mcoll.LineCollection(self.lines, **self.kwargs)


# Klasa Plot jest najważniejszą klasą w całym programie, ponieważ agreguje
# wszystkie przygotowane sceny, odpowiada za stworzenie wykresu i przechowuje
# referencje na przyciski, dzięki czemu nie będą one skasowane podczas tzw.
# garbage collectingu.
class Plot:
    def __init__(self, scenes=[Scene()], points=[], lines=[], json=None):
        if json is None:
            self.scenes = scenes
            if points or lines:
                self.scenes[0].points = points
                self.scenes[0].lines = lines
        else:
            self.scenes = [Scene([PointsCollection(pointsCol) for pointsCol in scene["points"]],
                                 [LinesCollection(linesCol) for linesCol in scene["lines"]])
                           for scene in js.loads(json)]

    # Ta metoda ma szczególne znaczenie, ponieważ konfiguruje przyciski i
    # wykonuje tym samym dość skomplikowaną logikę. Zauważmy, że konfigurując każdy
    # przycisk podajemy referencję na metodę obiektu _Button_callback, która
    # zostanie wykonana w momencie naciśnięcia.
    def __configure_buttons(self):
        plt.subplots_adjust(bottom=0.2)
        ax_prev = plt.axes([0.6, 0.05, 0.15, 0.075])
        ax_next = plt.axes([0.76, 0.05, 0.15, 0.075])
        ax_add_point = plt.axes([0.44, 0.05, 0.15, 0.075])
        ax_add_line = plt.axes([0.28, 0.05, 0.15, 0.075])
        ax_add_rect = plt.axes([0.12, 0.05, 0.15, 0.075])
        b_next = Button(ax_next, 'Następny')
        b_next.on_clicked(self.callback.next)
        b_prev = Button(ax_prev, 'Poprzedni')
        b_prev.on_clicked(self.callback.prev)
        b_add_point = Button(ax_add_point, 'Dodaj punkt')
        b_add_point.on_clicked(self.callback.add_point)
        b_add_line = Button(ax_add_line, 'Dodaj linię')
        b_add_line.on_clicked(self.callback.add_line)
        b_add_rect = Button(ax_add_rect, 'Dodaj figurę')
        b_add_rect.on_clicked(self.callback.add_rect)
        return [b_prev, b_next, b_add_point, b_add_line, b_add_rect]

    def add_scene(self, scene):
        self.scenes.append(scene)

    def add_scenes(self, scenes):
        self.scenes = self.scenes + scenes

    # Metoda toJson() odpowiada za zapisanie stanu obiektu do ciągu znaków w
    # formacie JSON.
    def toJson(self):
        return js.dumps([{"points": [np.array(pointCol.points).tolist() for pointCol in scene.points],
                          "lines": [linesCol.lines for linesCol in scene.lines]}
                         for scene in self.scenes])

        # Metoda ta zwraca punkty dodane w trakcie rysowania.

    def get_added_points(self):
        if self.callback:
            return self.callback.added_points
        else:
            return None

    # Metoda ta zwraca odcinki dodane w trakcie rysowania.
    def get_added_lines(self):
        if self.callback:
            return self.callback.added_lines
        else:
            return None

    # Metoda ta zwraca wielokąty dodane w trakcie rysowania.
    def get_added_figure(self):
        if self.callback:
            return self.callback.added_rects
        else:
            return None

    # Metoda ta zwraca punkty, odcinki i wielokąty dodane w trakcie rysowania
    # jako scenę.
    def get_added_elements(self):
        if self.callback:
            return Scene(self.callback.added_points, self.callback.added_lines + self.callback.added_rects)
        else:
            return None

    # Główna metoda inicjalizująca wyświetlanie wykresu.
    def draw(self):
        plt.close()
        fig = plt.figure()
        self.callback = _Button_callback(self.scenes)
        self.widgets = self.__configure_buttons()
        ax = plt.axes(autoscale_on=False)
        self.callback.set_axes(ax)
        fig.canvas.mpl_connect('button_press_event', self.callback.on_click)
        plt.show()
        self.callback.draw()

## Funkcje pomocnicze

In [17]:
# funkcja orienta zwraca wyznacznik macierzy postaci
#  | ax, ay, 1 |
#  | bx, by, 1 |
#  | cx, cy, 1 |
# wyznacznik ten definiuje po jakiej stronie (lewo / prawo)
# znajduje się punkt c względem wektora -ab->
def orient(a, b, c):
    return a[0] * b[1] + a[1] * c[0] + b[0] * c[1] - b[1] * c[0] - a[1] * b[0] - a[0] * c[1]


# maksymalna odległość między punktami, aby dwa punkty zostały uznane za jeden
EPS = 1e-9

## Wczytywanie zbioru odcinków od użytkownika

Wczytywanie odcinków odbywa się poprzez wczytanie odcinków z użyciem przycisku "Dodaj linię"
Zbiór odcinków musi być ogólny, tzn:
* 

In [64]:
input_plot = Plot(scenes=[Scene()])
input_plot.draw()

<IPython.core.display.Javascript object>

In [59]:
# funkcja odpowiada za pobranie wczytanych odcinków z
# wykresu (Plot)
def get_input_lines(p):
    lines = []
    for collection in p.get_added_lines():
        for line in collection.lines:
            if line[0][0] < line[1][0]:
                lines.append(line)
            else:
                lines.append([line[1], line[0]])

    return lines


input_lines = get_input_lines(input_plot)
for line in input_lines:
    print(line)

[(-0.040806451612903236, 0.031611519607843155), (0.036370967741935475, 0.03127450980392159)]
[(-0.021068548387096785, -0.009503676470588224), (0.020181451612903217, -0.02399509803921568)]
[(0.041028225806451615, 0.04408088235294119), (0.049899193548387094, 0.04408088235294119)]


In [ ]:
## HERE ZAPIS DO PLIKU ALE TO ŁATWE TO MI SIĘ NIE CHCE
## OK

## Struktury reprezentujące punkt, odcinek oraz ścianę (trapez/trójkąt)

In [71]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):  #do reprezentacji
        return f'{round(self.x, 4)} {round(self.y, 4)}'

    def __str__(self):  #zaokrąglam żeby było bardziej czytelne
        return f'{round(self.x, 4)} {round(self.y, 4)}'


class Segment:
    def __init__(self, p, q):
        self.p = p
        self.q = q
        if self.p.x > self.q.x:
            self.p, self.q = self.q, self.p
        # parametry reprezentujące prostą, do której należy odcinek
        # prosta ma równanie postaci y = ax + b
        self.a = (q.y - p.y) / (q.x - p.x)  # odcinki nie moga byc pionowe - nie ma problemu z dzieleniem przez 0
        self.b = self.p.y - (self.p.x * self.a)

    def __repr__(self):  #printuje punkty
        return f'{self.p} {self.q}'

    def __str__(self):
        return f'{self.p} {self.q}'

    # funkcja zwraca polozenie punktu wzgledem prostej
    # jesli punkt znajduje sie nad prosta zwraca liczbe dodatnia
    # wpp liczbe ujemna
    # jesli punkt lezy poza zakresem [p.x, q.x] - 0
    def point_position(self, point):
        if point.x < self.p.x or point.x > self.q.x:
            return 0
        return orient(self.p, self.q, point)

    # zwraca y = a * x + b jesli x miesci sie na odcinku
    # wpp zwraca None
    def y(self, x):
        if self.p.x > x or self.q.x < x:
            return None
        return self.a * x + self.b


class Trapezoid:  #klasa reprezentująca trapez (Nasza delta)
    def __init__(self):
        self.bottom = None
        self.top = None
        self.leftp = None
        self.rightp = None

        #Wskazniki na sąsiądów trapezu
        self.upper_right = None
        self.lower_right = None
        self.upper_left = None
        self.lower_left = None

        #Wskaznik do node w grafie
        self.node = None


class NodeArea:  #Klasa reprezentuje delte w grafie (przestrzeń) ewentualnie do poprawy
    def __init__(self, delta):
        self.delta = delta
        self.parents = [] #Przechowuje informacje o wszystkich Nodach którą są do niego podpiętę

class NodeVertex:  #Klasa reprezentuje wierzchołek w grafie (przestrzeń) ewentualnie do poprawy
    def __init__(self, left_side, right_side, vertex):
        self.left_side = left_side  #Pole przechowuje co się znajduje po lewej stronie wierzchołka
        self.right_side = right_side  #analogicznie
        self.vertex = vertex
        if isinstance(left_side, NodeArea): #zapisuje że area ma parenta
            left_side.parents.append(self)

        if isinstance(right_side, NodeArea): #analogicznie
            right_side.parents.append(self)


# z wykładu NodeX
class NodeSegment:  #Klasa reprezentuje odcinek w grafie (przestrzeń) ewentualnie do poprawy
    def __init__(self, upper_side, lower_side, segment):
        self.upper_side = upper_side  #Pole przechowuje co się znajduję nad odcinkiem
        self.lower_side = lower_side  #analogicznie
        self.segment = segment
        if isinstance(upper_side, NodeArea): #zapisuje że area ma parenta
            upper_side.parents.append(self)

        if isinstance(lower_side, NodeArea): #analogicznie
            lower_side.parents.append(self)


class Graph: #Jeszcze nie wiem co tu wsadzić
    def __init__(self, root):
        self.root = root
        root.parents.append(self) #zakładam że na początku zaczynamy z czystą przestrzenią



# Obsługa grafu

In [84]:
def find_in_graph(root, point): #Funkcja wyszukuje obszar na którym znajduje się dany Node
    if isinstance(root, NodeArea): #rozważany root jest obszarem
        return root.delta
    elif isinstance(root, NodeVertex): #rozważany root jest wierzchołkiem
        if point == root.vertex:
            pass
        elif point.x > root.vertex.x:
            return find_in_graph(root.right_side, point)
        else:
            return find_in_graph(root.left_side, point)
    elif isinstance(root, NodeSegment):#Rozważany root jest odcinkiem
        if point.y > root.segment.y(point.x):
            return find_in_graph(root.upper_side, point)
        else:
            return find_in_graph(root.lower_side, point)

def replace_in_node(delta_node, new_node): #Funkcja podmienia NodeArea czyli obszar w grafie na wierzchołki dla każdego z jego parentów
    for parent in delta_node.parents:
        if isinstance(parent, NodeVertex):
            if parent.left_side == delta_node:
                parent.left_side = new_node
            else:
                parent.right_side = new_node
        elif isinstance(parent, NodeSegment):
            if parent.upper_side == delta_node:
                parent.upper_side = new_node
            else:
                parent.lower_side = new_node
        elif isinstance(parent, Graph):
            parent.root = new_node

def make_new_area_mode(delta):
    new_node = NodeArea(delta)
    delta.node = new_node
    return  new_node

def update_one_delta(old_delta, segment, left_delta, right_delta, upper_delta, lower_delta):
    old_node = old_delta.node
    left_node = make_new_area_mode(left_delta)
    right_node = make_new_area_mode(right_delta)
    upper_node = make_new_area_mode(upper_delta)
    lower_node = make_new_area_mode(lower_delta)
    segment_node = NodeSegment(upper_node, lower_node, segment)
    q_node = NodeVertex(segment_node, right_node, segment.q)
    p_node = NodeVertex(left_node, q_node, segment.p)
    replace_in_node(old_node, p_node)

def update_more_delta():
    pass


In [85]:

def convert_to_segment(tuples_list): #Funkcja zamienia tablice krotek na tablice odcinków
    n = len(tuples_list)
    return [Segment(Point(elem[0][0], elem[0][1]), Point(elem[1][0], elem[1][1])) for elem in tuples_list]

print(convert_to_segment(input_lines))

def make_main_area(): #Funkcja tworzy przestrzeń będąca naszą mapą T
    main_delta = Trapezoid()
    main_delta.leftp = Point(-0.05, -0.05)
    main_delta.rightp = Point(0.05, 0.05)
    main_delta.bottom = Segment(main_delta.leftp, Point(0.05, -0.05))
    main_delta.top = Segment(main_delta.rightp, Point(-0.05, 0.05))
    return main_delta


#slajd 27
def get_delta_to_delete(graph, segment): #Funckja daje liste wszystkich przestrzeni które przechodzą przez odcinek
    delta = find_in_graph(graph.root, segment.p)
    all_delta = [delta]
    while delta.rightp.x < segment.q.x: #petla zbiera wszystkie delty, która so po drodzę do punktu q
        if delta.rightp.y > segment.y(delta.rightp.x):
            delta = delta.lower_right
        else:
            delta = delta.upper_right
        all_delta.append(delta)
    return all_delta

def border_delta_update(delta, segment, type): #TODO MORE COMPLICATED
    if type == "left":
        #Przestrzen na lewo od wierzchołka
        left_delta = Trapezoid()
        left_delta.leftp = delta.leftp
        left_delta.rightp = segment.p
        left_delta.top = delta.top
        left_delta.bottom = delta.bottom
        #Przestrzen na prawo-dół od wierzchołka
        lower_delta = Trapezoid()
        lower_delta.leftp = segment.p
        lower_delta.rightp = delta.rightp
        lower_delta.top = segment
        lower_delta.bottom = delta.bottom
        #Przestrzen na prawo-góra od wierzchołka
        upper_delta = Trapezoid()
        upper_delta.leftp = segment.q
        upper_delta.rightp = delta.rightp
        #to do poprawy

def update_T(all_delta, segment): #Przyjmuje zbiór delt oraz odcinek
    if len(all_delta) == 1: #Gdy mamy tylko jeden obszar do podzielenia
        delta = all_delta[0]
        #Gorna przestrzeń
        upper_delta = Trapezoid()
        upper_delta.top = delta.top
        upper_delta.bottom = segment
        upper_delta.leftp = segment.p
        upper_delta.rightp = Point(segment.q.x, segment.y(segment.q.x)) #punkt będący prawym narożnikiem dla górnej przestrzeni
        #Dolna przestrzeń
        lower_delta = Trapezoid()
        lower_delta.top = segment
        lower_delta.bottom = delta.bottom
        lower_delta.leftp =  Point(segment.p.x, segment.y(segment.p.x)) #punkt będący lewym dolnym narożnikiem dla dolnej przestrzeni
        lower_delta.rightp = segment.q
        #Prawa przestrzeń
        right_delta = Trapezoid()
        right_delta.top = delta.top
        right_delta.bottom = delta.bottom
        right_delta.leftp = segment.q
        right_delta.rightp = delta.rightp
        #Lewa przestrzeń
        left_delta = Trapezoid()
        left_delta.top = delta.top
        left_delta.bottom = delta.bottom
        left_delta.leftp = delta.leftp
        left_delta.rightp = segment.p
        #Aktualizacja grafu D
        update_one_delta(delta, segment, left_delta, right_delta, upper_delta, lower_delta)

    else: #Gdy mamy więcej niż jeden obszarów do podzielenia
        pass


def construct_map(): #Funkcja tworzy mape trapezoidalną czy chuj wie jak to nazwać
    T = make_main_area() #Stwórz przesteń T
    S = convert_to_segment(input_lines) #zamień krotki na listę odcinków
    T.node = NodeArea(T) #dodaj Node'a do niego
    graph = Graph(T.node) #stwórz graf i ustaw jako roota przestrzeń

    update_T(get_delta_to_delete(graph, S[0]), S[0])
    print(isinstance(graph.root,NodeVertex))


construct_map()



[-0.0408 0.0316 0.0364 0.0313, -0.0211 -0.0095 0.0202 -0.024, 0.041 0.0441 0.0499 0.0441]
True
